In [0]:
SET TIME ZONE 'Pacific/Auckland';

In [0]:
With L1 as (select TradingDate,TradingPeriodNumber, avg(DollarsPerMegawattHour) as AvgPrice 
from ea_prd.gold.factpredispatchandfinalenergyprices 
where TradingDate >= '1997-05-01' and PriceTypeCode = 'F'
group by TradingDate,TradingPeriodNumber)
select year(TradingDate)as YearID, month(TradingDate)  as MonthID, round(avg(AvgPrice),2) as AvgPrice  
from L1 group by YearID, MonthID
Order by YearID, MonthID

In [0]:
with HAY as (
  select year(TradingDate)as YearID, month(TradingDate)  as MonthID, round(avg(DollarsPerMegawattHour),2) as HAY_AvgPrice 
  from ea_prd.gold.factpredispatchandfinalenergyprices 
  where PointOfConnectionCode = 'HAY2201' and PriceTypeCode = 'F'
  and TradingDate >= '1996-10-01'
  group by YearID, MonthID)
, BEN as (
  select year(TradingDate)as YearID, month(TradingDate)  as MonthID, round(avg(DollarsPerMegawattHour),2) as BEN_AvgPrice 
  from ea_prd.gold.factpredispatchandfinalenergyprices 
  where PointOfConnectionCode = 'BEN2201' and PriceTypeCode = 'F'
  and TradingDate >= '1997-05-01'
  group by YearID, MonthID)
, OTA as (
  select year(TradingDate)as YearID, month(TradingDate)  as MonthID, round(avg(DollarsPerMegawattHour),2) as OTA_AvgPrice 
  from ea_prd.gold.factpredispatchandfinalenergyprices 
  where PointOfConnectionCode = 'OTA2201' and PriceTypeCode = 'F'
  and TradingDate >= '1996-10-01'
  group by YearID, MonthID)
select OTA.*, HAY_AvgPrice, BEN_AvgPrice from HAY 
inner join BEN on HAY.YearID = BEN.YearID and HAY.MonthID = BEN.MonthID
inner join OTA on HAY.YearID = OTA.YearID and HAY.MonthID = OTA.MonthID
Order by YearID, MonthID


In [0]:
With L1 as (
  Select year(pr.TradingDate)as YearID, month(pr.TradingDate)  as MonthID,
  round(sum(DollarsPerMegawattHour*KilowattHours/1000),2) as Dollars,
  round(sum(KilowattHours/1000),3) as MWh,
  round(sum(DollarsPerMegawattHour*KilowattHours)/sum(KilowattHours),2) as LWAP
  from ea_prd.gold.factpredispatchandfinalenergyprices pr
  inner join ea_prd.gold.factreconciledinjectionandofftake rec
  on pr.TradingDate = rec.TradingDate and pr.TradingPeriodNumber = rec.TradingPeriodNumber and pr.PointOfConnectionCode = rec.PointOfConnectionCode
  and pr.TradingDate >= '1997-05-01' and pr.PriceTypeCode = 'F' and rec.FlowDirection = 'Offtake' 
  GROUP BY year(pr.TradingDate), month(pr.TradingDate)
)
select * from L1 
Order by YearID, MonthID


Databricks visualization. Run in Databricks to view.